In [1]:
# import libraries
import pandas as pd
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import osmnx as ox
import json
from datetime import datetime, timedelta, date
import os
import time
import pyarrow
import sys
import sidetable
import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.tracebacklimit = 0 # turn off long tracebacks on error messages
#sys.tracebacklimit = None # use to turn tracebacks back on if needed

In [12]:
pd.set_option('display.max_columns', None)

In [17]:
nh_all=pd.read_parquet('data/nh_sub_Jun27_21_Jul10_22_concat.parquet.gzip')

In [18]:
nh_all.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,54,43,2,79.63,81.132,87,54,32,62.069,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,79,65,13,82.278,82.278,107,36,71,33.645,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,44,41,2,93.182,93.182,70,44,,62.857,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,68,64,,94.118,94.118,91,62,,68.132,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,79,67,,84.81,84.81,106,70,,66.038,66.038


In [19]:
nh_all.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
836807,2022-07-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,31,30,1,96.774,96.774,83,79,4,95.181,95.181
836808,2022-07-10,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,...,,,,,,,,,,
836809,2022-07-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,1,0,1,0,0,12,10,2,83.333,83.333
836810,2022-07-10,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,N,...,,,,,,,,,,
836811,2022-07-10,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [20]:
nh_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836812 entries, 0 to 836811
Data columns (total 32 columns):
 #   Column                                                                                                                                                          Non-Null Count   Dtype 
---  ------                                                                                                                                                          --------------   ----- 
 0   week_ending                                                                                                                                                     836812 non-null  object
 1   federal_provider_number                                                                                                                                         836812 non-null  object
 2   provider_name                                                                                                                   

In [21]:
#check for dup rows
nh_all.loc[nh_all.duplicated(),]

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time


In [22]:
# rename long cols
nh_all = nh_all.rename({'provider_zip_code':'provider_zip',
                                'passed_quality_assurance_check':'data_passed_qa',
                                'residents_weekly_confirmed_covid_19':'res_wk_confirmed_c19',
                                'weekly_resident_confirmed_covid_19_cases_per_1_000_residents':'wk_res_confirmed_c19_cases_per_1000',
                                'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result':'num_staff_new_pos_c19_test',
                                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week':'num_res_in_facility',
                                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_res_comp_c19_vax',
                                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_res_offered_declined_cvd_vax',
                                'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'pct_res_comp_c19_vax',
                                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week':'num_hcp',
                                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_hcp_comp_c19_vax',
                                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_hcp_offered_declined_c19_vax',
                                'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'pct_hcp_comp_c19_vax'
                               }, axis=1)

In [23]:
nh_all.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,...,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,54,43,2,79.63,81.132,87,54,32,62.069,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,79,65,13,82.278,82.278,107,36,71,33.645,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,44,41,2,93.182,93.182,70,44,,62.857,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,68,64,,94.118,94.118,91,62,,68.132,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,79,67,,84.81,84.81,106,70,,66.038,66.038


In [24]:
nh_all = nh_all[['week_ending','federal_provider_number', 'provider_name', 'provider_address', 'provider_city', 'provider_state', 'provider_zip', 'provider_phone_number', 'county', 'submitted_data','data_passed_qa','res_wk_confirmed_c19','wk_res_confirmed_c19_cases_per_1000','num_staff_new_pos_c19_test','num_res_in_facility','num_res_comp_c19_vax','num_res_offered_declined_cvd_vax','pct_res_comp_c19_vax','num_hcp','num_hcp_comp_c19_vax','num_hcp_offered_declined_c19_vax','pct_hcp_comp_c19_vax']]



In [ ]:
nh_all

## Shapefile data

In [3]:
# Read in shp file as dataframe
# Nursing home shapefile data source: HIFLD data retrieved from: 
# https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::nursing-homes/about
# Latest update for files: Jun 29, 22

nh_shp_df = gpd.read_file('data/Nursing_Homes_shape/NursingHomes.shp')
nh_shp_df.head()

,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,...,TOT_RES,TOT_STAFF,BEDS,EXCESS_BED,OWNERSHIP,MEDICAIDID,MEDICAREID,STATE_LIC,SOURCETYPE,geometry
0,1,0235,HAINES ASSISTED LIVING,219 UNION STREET,HAINES,AK,99827,NOT AVAILABLE,(907) 766-3616,ASSISTED LIVING,...,10,-999,10,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,100793,ASSISTED LIVING,POINT (-135.44587 59.23694)
1,2,0241,LAKEVIEW HOME,2675 WESLEYAN DRIVE,ANCHORAGE,AK,99508,NOT AVAILABLE,(907) 333-8921,ASSISTED LIVING,...,11,-999,11,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,235,ASSISTED LIVING,POINT (-149.78833 61.19643)
2,3,0257,ALASKA VETERANS' AND PIONEERS' HOME,250 E. FIREWEED AVENUE,PALMER,AK,99645,NOT AVAILABLE,(907) 745-4241,ASSISTED LIVING,...,79,-999,79,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,77,ASSISTED LIVING,POINT (-149.11363 61.59638)
3,4,0268,TRANQUILITY MANOR ESTATES,1950 HEMMER ROAD,PALMER,AK,99645,NOT AVAILABLE,(907) 746-4220,ASSISTED LIVING,...,17,-999,17,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,100954,ASSISTED LIVING,POINT (-149.14849 61.60133)
4,5,0249,NORTHERN COMFORT,2800 N. LAGOON DRIVE,WASILLA,AK,99654,NOT AVAILABLE,(907) 746-6493,ASSISTED LIVING,...,16,-999,16,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,241,ASSISTED LIVING,POINT (-149.28398 61.61176)
